In [1]:
#===========================================================================================
# ALBERT MASKED 예시
#
# AlbertTokenizerFast
# => AlbertTokenizerFast 인데 한국어도 토큰화되도록 하려면, tokenizer_config.json에 masked_token-normalized:true 로 해줘야함.
# => 그렇지 않으면 한글 단어들은 모두 [unk] 으로 되어 버림 인데 
# 
#===========================================================================================

import torch
from transformers import AlbertTokenizer, AlbertTokenizerFast, AutoTokenizer, BertTokenizerFast
from tqdm.notebook import tqdm

do_lower_case_ = True # 1=영어 대문자를 소문자로 변경, 0=대.소문자 구분
keep_acccents_ = False

# 기본 <unk>, <pad> 인데, 변경된 경우에는 아래 값을 변경해주면됨
unk_t ='[UNK]' #UNK 토큰 = <unk>
pad_t ='[PAD]' #PAD 토큰 = <pad>

model_path = '../../data11/ai_hub/vocab/tl1-1줄-mecab-30000-sp-unigram-22M-vocab'

#tokenizer = AlbertTokenizer.from_pretrained(model_path, max_len=128, do_lower_case=do_lower_case_, unk_token=unk_t, pad_token=pad_t)
tokenizer = AlbertTokenizerFast.from_pretrained(model_path, max_len=128, do_lower_case=do_lower_case_, keep_acccents=keep_acccents_, nk_token=unk_t, pad_token=pad_t)
#tokenizer = BertTokenizerFast.from_pretrained(model_path, max_len=128, do_lower_case=do_lower_case_param)

print(f'{model_path} is_fast:{tokenizer.is_fast}')
print(len(tokenizer))


../../data11/ai_hub/vocab/tl1-1줄-mecab-30000-sp-unigram-22M-vocab is_fast:True
30000


In [3]:
# Be aware, overflowing tokens are not returned for the setting you have chosen, 
# i.e. sequence pairs with the 'longest_first' truncation strategy. 
# So the returned list will always be empty even if some tokens have been removed.

sentence_a = "I love you. "
sentence_b = "난 널 사랑해.충무공 이순신은 조선의 최고의 장수이다.모코엠시스에서는 문서중앙화 및 보안파일서버 솔루션인 엠파워를 출시하였다."

result = tokenizer(sentence_a, sentence_b, max_length=32, padding=True, truncation=True, return_overflowing_tokens=False)

# tokenizer 출력
print(result)

print(tokenizer.decode(result.input_ids))

# 토큰별루 출력
for idx in range(len(result.input_ids)):
    print(tokenizer.decode(result.input_ids[idx]))
    if tokenizer.decode(result.input_ids[idx]) == '[PAD]':
        break
        
        
# 각 스페셜 tokenid를 구함
CLStokenid = tokenizer.convert_tokens_to_ids('[CLS]')
SEPtokenid = tokenizer.convert_tokens_to_ids('[SEP]')
UNKtokenid = tokenizer.convert_tokens_to_ids('<UNK>')
PADtokenid = tokenizer.convert_tokens_to_ids('<pad>')
MASKtokenid = tokenizer.convert_tokens_to_ids('[MASK]')

print(CLStokenid)
print(SEPtokenid)
print(UNKtokenid)
print(PADtokenid)
print(MASKtokenid)

{'input_ids': [5, 2534, 7333, 7260, 6676, 39, 6, 1619, 3786, 515, 1663, 6676, 26497, 12, 2662, 1775, 1665, 1280, 1510, 319, 1510, 6361, 251, 670, 6676, 1639, 2422, 10449, 224, 3518, 326, 6], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}
[CLS] i love you. [SEP] 난 널 사랑해.충무공 이순신은 조선의 최고의 장수이다.모코엠시스에서[SEP]
[CLS]
i
love
you
.

[SEP]
난
널
사랑
해
.
충무공
이
순
신
은
조선
의
최고
의
장수
이
다
.
모
코
엠
시스
에
서
[SEP]
5
6
4
4
7


In [ ]:
# masked 예시

from transformers import AlbertForMaskedLM
import torch
import torch.nn.functional as F

model = AlbertForMaskedLM.from_pretrained(model_path)
text = ['한국 수도는 [MASK] 이다', '프랑스 수도는 [MASK]이다', '충무공 이순신은 [MASK]에 최고의 장수였다']
tokenized_input = tokenizer(text, max_length=128, truncation=True, padding='max_length', return_tensors='pt')
outputs = model(**tokenized_input)
logits = outputs.logits
mask_idx_list = []
for tokens in tokenized_input['input_ids'].tolist():
    token_str = [tokenizer.convert_ids_to_tokens(s) for s in tokens]
    
    # **위 token_str리스트에서 [MASK] 인덱스를 구함
    # => **해당 [MASK] 안덱스 값 mask_idx 에서는 아래 출력하는데 사용됨
    mask_idx = token_str.index('[MASK]')
    mask_idx_list.append(mask_idx)
    
for idx, mask_idx in enumerate(mask_idx_list):
    
    logits_pred=torch.argmax(F.softmax(logits[idx]), dim=1)
    mask_logits_idx = int(logits_pred[mask_idx])
    # [MASK]에 해당하는 token 구함
    mask_logits_token = tokenizer.convert_ids_to_tokens(mask_logits_idx)
    # 결과 출력 
    print('\n')
    print('*Input: {}'.format(text[idx]))
    print('*[MASK] : {} ({})'.format(mask_logits_token, mask_logits_idx))